##Team 5 Jupyter notebook - Project 1

In [13]:
#Dependencies
import pandas as pd
import numpy as np
import requests
import random
import gmaps
import json
import os

In [ ]:
## Create a url API INEGI
url_path = https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/668921,6207061405/es//true/BIE/2.0/1eda5bfc-0e53-ec67-9976-42321b74e950?type=json

In [15]:
import pandas as pd
a = pd.read_excel("Resources/Centraleselectricas-2.xlsx")
a

,No.,Número de identificación del proyecto asignado por el distribuidor,Nivel de tensión en el punto de interconexión,Número de fases,Capacidad de la central eléctrica (kW),Área Administrativa del Distribuidor (Division),Área Administrativa del Distribuidor (Zona),Estado,Municipio,Fecha Solicitud,Estado en que se encuentra la solicitud: Recibida/Atendida/En lista de espera/Cancelada,Fecha de Interconexión Física,Estudio de interconexión requerido (Si/No),Tecnología,Ubicación por circuito de distribución,Capacidad del circuito de distribución para albergar CE de GD,Ubicación por subestación,Ubicación por Red de Distribución,Ubicación por NodoP,Regímen de contraprestación seleccionado
0,1,62003,BAJA,2,1.65,DIVISION BAJA CALIFORNIA,ZONA TIJUANA,Baja California,Tijuana,2017-05-29,ATENDIDA,2017-05-31,No,FUENTE DE ENERGIA SOLAR,RII04175,No disponible,RIO,No disponible,07RII-69,MEDICIÓN NETA DE ENERGÍA
1,2,64518,BAJA,2,4.88,DIVISION BAJA CALIFORNIA,ZONA TIJUANA,Baja California,Tijuana,2017-04-13,ATENDIDA,2017-04-19,No,FUENTE DE ENERGIA SOLAR,HMO04155,No disponible,HIPODROMO,No disponible,07HMO-69,MEDICIÓN NETA DE ENERGÍA
2,3,63639,BAJA,2,3.72,DIVISION BAJA CALIFORNIA,ZONA TIJUANA,Baja California,Tijuana,2017-05-02,ATENDIDA,2017-05-11,No,FUENTE DE ENERGIA SOLAR,HMO04155,No disponible,HIPODROMO,No disponible,07HMO-69,MEDICIÓN NETA DE ENERGÍA
3,4,63640,BAJA,2,5.58,DIVISION BAJA CALIFORNIA,ZONA TIJUANA,Baja California,Tijuana,2017-04-20,ATENDIDA,2017-04-24,No,FUENTE DE ENERGIA SOLAR,HMO04155,No disponible,HIPODROMO,No disponible,07HMO-69,MEDICIÓN NETA DE ENERGÍA
4,5,66728,BAJA,2,3.72,DIVISION BAJA CALIFORNIA,ZONA TIJUANA,Baja California,Tijuana,2017-04-21,ATENDIDA,2017-04-25,No,FUENTE DE ENERGIA SOLAR,HMO04155,No disponible,HIPODROMO,No disponible,07HMO-69,MEDICIÓN NETA DE ENERGÍA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100332,100333,196291,BAJA,2,1.50,DIVISION VALLE MEXICO SUR ...,ZONA VOLCANES,Estado de México,Amanalco,2019-11-26,EN LISTA DE ESPERA,2019-12-03,NO,FUENTE DE ENERGIA SOLAR ...,IXL53085,2.62,IXTAPALUCA,No Disponible,01IXL-230,MEDICIÓN NETA DE ENERGÍA ...
100333,100334,195138,BAJA,2,2.64,DIVISION VALLE MEXICO SUR ...,ZONA VOLCANES,Estado de México,Calimaya,2019-08-29,EN LISTA DE ESPERA,2019-09-05,NO,FUENTE DE ENERGIA SOLAR ...,JCP53035,1.82,JUCHITEPEC,No Disponible,01JCP-85,MEDICIÓN NETA DE ENERGÍA ...
100334,100335,213873,BAJA,2,1.00,DIVISION VALLE MEXICO SUR ...,ZONA VOLCANES,Estado de México,Tenango del Aire,2019-12-20,EN LISTA DE ESPERA,2019-12-27,NO,FUENTE DE ENERGIA SOLAR ...,JCP53025,2.02,JUCHITEPEC,No Disponible,01JCP-85,MEDICIÓN NETA DE ENERGÍA ...
100335,100336,202369,BAJA,2,9.00,DIVISION VALLE MEXICO SUR ...,ZONA VOLCANES,Estado de México,Tlalmanalco,2019-07-23,EN LISTA DE ESPERA,2019-07-30,NO,FUENTE DE ENERGIA SOLAR ...,AGV53015,3.31,AGUA VIVA,No Disponible,01AGV-85,MEDICIÓN NETA DE ENERGÍA ...


In [2]:
#Data cleaning

In [3]:
#API's

In [4]:
#Ploatings